## DeepLabV3 Semantic segmentation on Cityscapes dataset
Based on:
https://github.com/fregu856/deeplabv3/tree/master

Others examples:
- https://github.com/CoinCheung/DeepLab-v3-plus-cityscapes
- https://github.com/VainF/DeepLabV3Plus-Pytorch?tab=readme-ov-file
- https://github.com/chenxi116/DeepLabv3.pytorch
- https://github.com/giovanniguidi/deeplabV3-PyTorch/tree/master

### Environment Pytorch39

In [1]:
import sys,os
current_path=os.getcwd()

# imported from datasets.py
from datasets import DatasetTrain, DatasetVal # (this needs to be imported before torch, because cv2 needs to be imported before torch for some reason)

# imported from /model/deeplabv3.py
sys.path.append(current_path+"/model")
from deeplabv3 import DeepLabV3

# imported from /utils/utils.py
sys.path.append(current_path+"/utils")
from utils import add_weight_decay

import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import cv2

import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
# NOTE! NOTE! change this to not overwrite all log data when you train the model:
model_id = "1"

num_epochs = 300
batch_size = 3
learning_rate = 0.0001

network = DeepLabV3(model_id, project_dir=current_path).cuda()

train_dataset = DatasetTrain(cityscapes_data_path=current_path+"/data/cityscapes",
                             cityscapes_meta_path=current_path+"/data/cityscapes/meta")
val_dataset = DatasetVal(cityscapes_data_path=current_path+"/data/cityscapes",
                         cityscapes_meta_path=current_path+"/data/cityscapes/meta")

num_train_batches = int(len(train_dataset)/batch_size)
num_val_batches = int(len(val_dataset)/batch_size)
print ("num_train_batches:", num_train_batches)
print ("num_val_batches:", num_val_batches)

pretrained resnet, 18
num_train_batches: 991
num_val_batches: 166


In [3]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, shuffle=True,
                                           num_workers=1)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=batch_size, shuffle=False,
                                         num_workers=1)

In [4]:
params = add_weight_decay(network, l2_value=0.0001)
optimizer = torch.optim.Adam(params, lr=learning_rate)

with open(current_path+"/data/cityscapes/meta/class_weights.pkl", "rb") as file: # (needed for python3)    
    class_weights = np.array(pickle.load(file))
class_weights = torch.from_numpy(class_weights)
class_weights = Variable(class_weights.type(torch.FloatTensor)).cuda()

# loss function
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

## Train

In [5]:
epoch_losses_train = []
epoch_losses_val = []
for epoch in range(num_epochs):
    
    print ("###########################")
    print ("epoch: %d/%d" % (epoch+1, num_epochs))

    ############################################################################
    # train:
    ############################################################################
    network.train() # (set in training mode, this affects BatchNorm and dropout)
    batch_losses = []
    current_time = time.time()
    for step, (imgs, label_imgs) in enumerate(train_loader):
        #current_time = time.time()

        imgs = Variable(imgs).cuda() # (shape: (batch_size, 3, img_h, img_w))
        label_imgs = Variable(label_imgs.type(torch.LongTensor)).cuda() # (shape: (batch_size, img_h, img_w))

        outputs = network(imgs) # (shape: (batch_size, num_classes, img_h, img_w))

        # compute the loss:
        loss = loss_fn(outputs, label_imgs)
        loss_value = loss.data.cpu().numpy()
        batch_losses.append(loss_value)

        # optimization step:
        optimizer.zero_grad() # (reset gradients)
        loss.backward()       # (compute gradients)
        optimizer.step()      # (perform optimization step)

    print (time.time() - current_time)

    epoch_loss = np.mean(batch_losses)
    epoch_losses_train.append(epoch_loss)
    with open("%s/epoch_losses_train.pkl" % network.model_dir, "wb") as file:
        pickle.dump(epoch_losses_train, file) 
        
    print ("train loss: %g" % epoch_loss)
    plt.figure(1)
    plt.plot(epoch_losses_train, "k^")
    plt.plot(epoch_losses_train, "k")
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.title("train loss per epoch")
    plt.savefig("%s/epoch_losses_train.png" % network.model_dir)
    plt.close(1)

    print ("####")


###########################
epoch: 1/300
223.29293704032898
train loss: 1.84293
####
###########################
epoch: 2/300
221.00732707977295
train loss: 1.43343
####
###########################
epoch: 3/300
218.3480622768402
train loss: 1.31758
####
###########################
epoch: 4/300
221.02988719940186
train loss: 1.24397
####
###########################
epoch: 5/300
220.00473594665527
train loss: 1.19078
####
###########################
epoch: 6/300
218.4487748146057
train loss: 1.10872
####
###########################
epoch: 7/300
218.63233542442322
train loss: 1.10276
####
###########################
epoch: 8/300
219.36339259147644
train loss: 1.03531
####
###########################
epoch: 9/300
219.26627254486084
train loss: 1.02967
####
###########################
epoch: 10/300
219.81028771400452
train loss: 1.03096
####
###########################
epoch: 11/300
217.8560974597931
train loss: 0.974849
####
###########################
epoch: 12/300
218.96096324920654
trai

## Validation

In [6]:
    ############################################################################
    # val:
    ############################################################################
    network.eval() # (set in evaluation mode, this affects BatchNorm and dropout)
    batch_losses = []
    for step, (imgs, label_imgs, img_ids) in enumerate(val_loader):
        with torch.no_grad(): # (corresponds to setting volatile=True in all variables, this is done during inference to reduce memory consumption)
            imgs = Variable(imgs).cuda() # (shape: (batch_size, 3, img_h, img_w))
            label_imgs = Variable(label_imgs.type(torch.LongTensor)).cuda() # (shape: (batch_size, img_h, img_w))

            outputs = network(imgs) # (shape: (batch_size, num_classes, img_h, img_w))

            # compute the loss:
            loss = loss_fn(outputs, label_imgs)
            loss_value = loss.data.cpu().numpy()
            batch_losses.append(loss_value)

    epoch_loss = np.mean(batch_losses)
    epoch_losses_val.append(epoch_loss)
    with open("%s/epoch_losses_val.pkl" % network.model_dir, "wb") as file:
        pickle.dump(epoch_losses_val, file)
    print ("val loss: %g" % epoch_loss)
    plt.figure(1)
    plt.plot(epoch_losses_val, "k^")
    plt.plot(epoch_losses_val, "k")
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.title("val loss per epoch")
    plt.savefig("%s/epoch_losses_val.png" % network.model_dir)
    plt.close(1)

    # save the model weights to disk:
    checkpoint_path = network.checkpoints_dir + "/model_" + model_id +"_epoch_" + str(epoch+1) + ".pth"
    torch.save(network.state_dict(), checkpoint_path)


val loss: 0.580649
